# Раздел 1 — Сбор текстов (VK / сайты / Яндекс Отзывы)

В этом разделе мы приводим тексты из разных источников к **единому формату** таблицы `docs`.

## Ограничения курса
- Используем **только тексты**.
- **LLM запрещены**.
- Никаких внешних баз, требующих предварительной разметки.
- Допустимо: классические библиотеки + BERT/эмбеддинги и методы на их основе (в следующих разделах).

## Единая схема данных `docs`
- `doc_id`: уникальный ID документа
- `source`: `vk | website | yandex_reviews`
- `text_raw`: исходный текст
- `text_clean`: очищенный текст (в этом разделе — простая базовая очистка)
- `date`: дата (если есть)
- `url`: ссылка на источник (если есть)
- `meta`: словарь с доп. полями (например, группа VK, рейтинг, заголовок)


In [ ]:
from __future__ import annotations

import re
from typing import Any, Dict, List, Optional

import numpy as np
import pandas as pd

import ipywidgets as widgets
from IPython.display import display, Markdown


In [ ]:
import importlib.util

_is_colab = importlib.util.find_spec('google.colab') is not None
if _is_colab:
    from google.colab import output  # type: ignore
    output.enable_custom_widget_manager()


## Панель управления
Выберите источник и введите параметры. Для демонстрации можно нажать **«Загрузить DEMO-корпус»**.

In [ ]:
#@title Форма управления { display-mode: "form" }
source_dd = widgets.Dropdown(
    options=[("VK (стены групп)", "vk"), ("Сайты", "website"), ("Яндекс Отзывы", "yandex_reviews")],
    value="vk",
    description="Источник:",
    style={"description_width": "initial"},
    layout=widgets.Layout(width="420px"),
)

input_ta = widgets.Textarea(
    value="",
    placeholder="Для VK: ID групп через запятую\nДля сайтов: URL-ы по строкам\nДля Яндекс Отзывов: URL-ы по строкам",
    description="Ввод:",
    style={"description_width": "initial"},
    layout=widgets.Layout(width="820px", height="140px"),
)

since_txt = widgets.Text(
    value="",
    placeholder="YYYY-MM-DD (необязательно)",
    description="Период с:",
    style={"description_width": "initial"},
    layout=widgets.Layout(width="260px"),
)

until_txt = widgets.Text(
    value="",
    placeholder="YYYY-MM-DD (необязательно)",
    description="Период по:",
    style={"description_width": "initial"},
    layout=widgets.Layout(width="260px"),
)

demo_btn = widgets.Button(description="Загрузить DEMO-корпус", button_style="info")
run_btn = widgets.Button(description="Запустить сбор (stub)", button_style="primary")

out = widgets.Output()

display(widgets.VBox([
    source_dd,
    input_ta,
    widgets.HBox([since_txt, until_txt, demo_btn, run_btn]),
    out,
]))


## Базовая очистка текста
Пока что делаем минимально:
- убираем HTML-теги,
- нормализуем пробелы,
- схлопываем повторяющиеся знаки.

Более серьёзная нормализация (леммы, дедупликация и т.д.) — в следующем разделе «Подготовка корпуса».

In [ ]:
HTML_TAG_RE = re.compile(r"<[^>]+>")
SPACE_RE = re.compile(r"\s+")
PUNCT_RUN_RE = re.compile(r"([!?.,])\1{2,}")

def clean_text_minimal(text: str) -> str:
    if text is None:
        return ""
    t = str(text)
    t = HTML_TAG_RE.sub(" ", t)
    t = PUNCT_RUN_RE.sub(r"\1\1", t)
    t = SPACE_RE.sub(" ", t).strip()
    return t

def parse_date_safe(s: str) -> Optional[pd.Timestamp]:
    s = (s or "").strip()
    if not s:
        return None
    try:
        return pd.to_datetime(s)
    except Exception:
        return None


## Stub-функции источников
Здесь **точки подключения** к реальным парсерам.

✅ VK — у тебя уже есть (сюда просто подключим функцию).  
🧩 Сайты и Яндекс Отзывы — позже добавим реальные парсеры.

Сейчас эти функции возвращают маленький тестовый набор, чтобы остальной pipeline работал.

In [ ]:
def collect_vk_stub(group_ids: List[str], since: Optional[pd.Timestamp], until: Optional[pd.Timestamp]) -> pd.DataFrame:
    rows = [
        {
            "doc_id": "vk_1",
            "source": "vk",
            "text_raw": "Люблю этот район — здесь тихо и много зелени. Но парковки не хватает!!!",
            "date": pd.Timestamp("2025-09-12"),
            "url": None,
            "meta": {"group_id": group_ids[0] if group_ids else None},
        },
        {
            "doc_id": "vk_2",
            "source": "vk",
            "text_raw": "Опять перекопали улицу у станции. Дойти до остановки — квест.",
            "date": pd.Timestamp("2025-10-03"),
            "url": None,
            "meta": {"group_id": group_ids[0] if group_ids else None},
        },
    ]
    return pd.DataFrame(rows)

def collect_websites_stub(urls: List[str]) -> pd.DataFrame:
    rows = [
        {
            "doc_id": "web_1",
            "source": "website",
            "text_raw": "<article>Исторический квартал меняется: появляются новые кафе и мастерские.</article>",
            "date": None,
            "url": urls[0] if urls else None,
            "meta": {"title": "Заглушка статьи"},
        }
    ]
    return pd.DataFrame(rows)

def collect_yandex_reviews_stub(urls: List[str]) -> pd.DataFrame:
    rows = [
        {
            "doc_id": "ya_1",
            "source": "yandex_reviews",
            "text_raw": "Удобно добираться, но внутри тесно. Персонал норм.",
            "date": None,
            "url": urls[0] if urls else None,
            "meta": {"rating": 3, "place": "Заглушка"},
        }
    ]
    return pd.DataFrame(rows)

def standardize_docs(df: pd.DataFrame) -> pd.DataFrame:
    required_cols = ["doc_id", "source", "text_raw", "date", "url", "meta"]
    for c in required_cols:
        if c not in df.columns:
            df[c] = None
    df = df[required_cols].copy()
    df["text_clean"] = df["text_raw"].map(clean_text_minimal)
    return df


## Запуск (DEMO / stub)
Ниже — обработчики кнопок. На DEMO можно ориентироваться как на ожидаемый формат результата.

In [ ]:
docs: Optional[pd.DataFrame] = None

def get_inputs() -> Dict[str, Any]:
    src = source_dd.value
    raw = input_ta.value.strip()
    since = parse_date_safe(since_txt.value)
    until = parse_date_safe(until_txt.value)

    if src == "vk":
        items = [x.strip() for x in re.split(r"[\n,;]+", raw) if x.strip()]
    else:
        items = [x.strip() for x in raw.splitlines() if x.strip()]

    return {"source": src, "items": items, "since": since, "until": until}

def load_demo(_=None):
    global docs
    with out:
        out.clear_output()
        demo = pd.concat([
            collect_vk_stub(["demo_group"], None, None),
            collect_websites_stub(["https://example.com/article"]),
            collect_yandex_reviews_stub(["https://example.com/reviews"]),
        ], ignore_index=True)
        docs = standardize_docs(demo)
        display(Markdown("✅ Загружен DEMO-корпус. Ниже — первые строки `docs`."))
        display(docs.head(10))

def run_stub(_=None):
    global docs
    cfg = get_inputs()
    with out:
        out.clear_output()
        display(Markdown(
            f"**Источник:** `{cfg['source']}`  \\n"
            f"**Элементы ввода:** {len(cfg['items'])}  \\n"
            f"**Период:** {cfg['since']} — {cfg['until']}"
        ))

        if cfg["source"] == "vk":
            df = collect_vk_stub(cfg["items"], cfg["since"], cfg["until"])
        elif cfg["source"] == "website":
            df = collect_websites_stub(cfg["items"])
        else:
            df = collect_yandex_reviews_stub(cfg["items"])

        docs = standardize_docs(df)
        display(Markdown("✅ Получена таблица `docs` (в формате для дальнейших разделов)."))
        display(docs)

demo_btn.on_click(load_demo)
run_btn.on_click(run_stub)


## Быстрый контроль качества корпуса (минимальный)
Это не “полный препроцессинг”, а sanity-check:
- сколько документов,
- распределение длины,
- топ-слова (очень грубо).

In [ ]:
def quick_qc(docs: pd.DataFrame) -> None:
    if docs is None or docs.empty:
        display(Markdown("⚠️ `docs` пустой. Сначала загрузите DEMO или выполните сбор."))
        return

    df = docs.copy()
    df["len_chars"] = df["text_clean"].map(lambda s: len(s or ""))
    df["len_words"] = df["text_clean"].map(lambda s: len((s or "").split()))

    display(Markdown("### Сводка"))
    display(df.groupby("source").agg(
        n_docs=("doc_id", "count"),
        avg_words=("len_words", "mean"),
        p50_words=("len_words", "median"),
        avg_chars=("len_chars", "mean"),
    ))

    display(Markdown("### Длина текста (слов)"))
    import matplotlib.pyplot as plt
    plt.figure()
    df["len_words"].hist(bins=20)
    plt.xlabel("words")
    plt.ylabel("count")
    plt.show()

    display(Markdown("### Топ-слов (очень грубо)"))
    token_re = re.compile(r"[а-яё]+", re.IGNORECASE)
    tokens = []
    for t in df["text_clean"].astype(str).tolist():
        tokens.extend(token_re.findall(t.lower()))
    if not tokens:
        display(Markdown("(нет токенов после очистки)"))
        return
    vc = pd.Series(tokens).value_counts().head(25)
    display(vc.to_frame("count"))

if docs is not None:
    quick_qc(docs)


## Что дальше
Следующий раздел книги: **«Подготовка корпуса»** — там будет:
- более строгая очистка,
- дедупликация,
- подготовка к эмбеддингам и тематизации.